In [1]:
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.0 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import keras
import keras_tuner
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

import os
from google.colab import drive

In [3]:
#Globals
num_features = 595
num_classes = 4 # no punch, straight, hook, uppercut
NUM_FEATURES_PER_SEQUENCE = 98

In [4]:
#Loading data

drive.mount('/content/drive')
folderPath = "/content/drive/MyDrive/Year 2/229 ._./Labeled/" #change as appropriate
file_list = [f for f in os.listdir(folderPath)]
Xs = []
Ys = []

dtype = [('column1', 'U10')] + [('column{}'.format(i), float) for i in range(2, num_features + 1)]

for file in file_list:
  print(file)
  temp = np.genfromtxt(f"{folderPath}{file}", delimiter=',', dtype=dtype)
  data = np.array([list(row) for row in temp])
  Xs.append(np.array([row[7:].astype(float) for row in data]))
  Ys.append(np.array([row[6].astype(float).astype(int) for row in data]))

X_nonorm = np.vstack(Xs)
mean = np.mean(X_nonorm, axis=0)
std_dev = np.std(X_nonorm, axis=0)
normalized_data = (X_nonorm - mean) / std_dev
Y = np.hstack(Ys)

Mounted at /content/drive
hagler_hearns_labeled_features.csv
OneOfGreatest_labeled_features.csv
spencer_crawford_labeled_training.csv
diaz_marquez_labeled_features.csv
rodriguez_gabello_labeled_features.csv
bivol_makhteinko_labeled_features.csv
irish_mango_labeled.csv
kom_garcia_labeled.csv
mayer_baumgarnder_labeled.csv
corrales_castillo_labeled.csv
bronner_maidana_labeled.csv
cotto_martinez_labeled.csv
diaz_marquez2_labeled.csv
mayer_hamadouche_labeled.csv
taylor_serrano_labeled.csv
froch_taylor_labeled.csv
cameron_taylor_labeled.csv


In [5]:
X = normalized_data
numSamples = len(X)
X = np.reshape(X, (numSamples, 6, NUM_FEATURES_PER_SEQUENCE))
# #80 10 10 train test val
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
# X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
# y_val_onehot = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)
# y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
from sklearn.model_selection import StratifiedShuffleSplit

sss_train_test = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
sss_train_val = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)

for train_index, test_index in sss_train_test.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

for test_index, val_index in sss_train_val.split(X_test, y_test):
    X_val, X_test = X_test[val_index], X_test[test_index]
    y_val, y_test = y_test[val_index], y_test[test_index]

y_val_onehot = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)
y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)

class_counts = tf.math.bincount(Y)
weights = {class_label: numSamples / count for class_label, count in enumerate(class_counts)}

In [ ]:
#HP Search
def balanced_acc(y_true, y_pred):
    y_pred_labels = tf.argmax(y_pred, axis=-1)
    y_true_labels = tf.argmax(y_true, axis=-1)
    return tf.py_function(balanced_accuracy_score, (y_true_labels, y_pred_labels), tf.float64)

def build_model(hp):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32), input_shape=(6, NUM_FEATURES_PER_SEQUENCE), return_sequences=True)))
    model.add(tf.keras.layers.Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=hp.Int('units2', min_value=32, max_value=512, step=32), input_shape=(6, NUM_FEATURES_PER_SEQUENCE))))
    model.add(tf.keras.layers.Dropout(rate=hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    hp_lr = hp.Choice('learning_rate', values=[0.0001, 0.00025, 0.0005, 0.00075, 0.001])
    hp_bs = hp.Choice('batch_size', values=[32, 64, 128, 256, 512])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr),
                  loss='categorical_crossentropy',
                  metrics=[balanced_acc])
    return model

tuner = RandomSearch(
    build_model,
    objective=keras_tuner.Objective("val_balanced_acc", direction="max"),
    max_trials=50,
    executions_per_trial=1,
    project_name='got_em_all'
    )

tuner.search(X_train, y_train_onehot, epochs=5, validation_data=(X_val, y_val_onehot))

Trial 13 Complete [00h 27m 10s]
val_balanced_acc: 0.5018725991249084

Best val_balanced_acc So Far: 0.5409830808639526
Total elapsed time: 03h 23m 37s

Search: Running Trial #14

Value             |Best Value So Far |Hyperparameter
480               |96                |units
0.2               |0.3               |dropout
384               |128               |units2
0.4               |0.2               |dropout2
0.001             |0.00075           |learning_rate
128               |64                |batch_size

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


  6/722 [..............................] - ETA: 3:50 - loss: 0.9902 - balanced_acc: 0.3276

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


 15/722 [..............................] - ETA: 3:50 - loss: 0.7160 - balanced_acc: 0.3856

...

262/722 [=========>....................] - ETA: 2:55 - loss: 0.2729 - balanced_acc: 0.5834

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


269/722 [==========>...................] - ETA: 2:52 - loss: 0.2727 - balanced_acc: 0.5822

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


272/722 [==========>...................] - ETA: 2:50 - loss: 0.2732 - balanced_acc: 0.5833

In [15]:
best_hps = tuner.get_best_hyperparameters(num_trials=10)
print("Top 10 Sets of Hyperparameters:")
for i, hyperparameters in enumerate(best_hps, 1):
    print(f"Top {i}:", hyperparameters.values)

Top 10 Sets of Hyperparameters:
Top 1: {'units': 448, 'dropout': 0.0, 'units2': 96, 'dropout2': 0.4, 'learning_rate': 0.00075, 'batch_size': 64}
Top 2: {'units': 352, 'dropout': 0.1, 'units2': 416, 'dropout2': 0.30000000000000004, 'learning_rate': 0.001, 'batch_size': 256}
Top 3: {'units': 160, 'dropout': 0.1, 'units2': 32, 'dropout2': 0.1, 'learning_rate': 0.00075, 'batch_size': 64}
Top 4: {'units': 384, 'dropout': 0.2, 'units2': 320, 'dropout2': 0.0, 'learning_rate': 0.00075, 'batch_size': 64}
Top 5: {'units': 384, 'dropout': 0.0, 'units2': 32, 'dropout2': 0.30000000000000004, 'learning_rate': 0.001, 'batch_size': 256}
Top 6: {'units': 416, 'dropout': 0.0, 'units2': 352, 'dropout2': 0.1, 'learning_rate': 0.0005, 'batch_size': 128}
Top 7: {'units': 96, 'dropout': 0.1, 'units2': 352, 'dropout2': 0.1, 'learning_rate': 0.001, 'batch_size': 512}
Top 8: {'units': 448, 'dropout': 0.2, 'units2': 160, 'dropout2': 0.2, 'learning_rate': 0.001, 'batch_size': 128}
Top 9: {'units': 416, 'dropout':

In [13]:
all_trials = tuner.oracle.get_best_trials(num_trials=len(tuner.oracle.trials))

# Extract validation loss values for all trials
#print(f"POOP: {all_trials[1].metrics.metrics['val_balanced_acc'].get_best_value()}")

loss_values = [trial.metrics.metrics['loss'].get_best_value() for trial in all_trials]
print(loss_values)

# Plot the graph
plt.figure(figsize=(10, 6))
plt.plot(loss_values, marker='o', linestyle='-')
plt.xlabel('Trial')
plt.ylabel('Validation Loss')
plt.title('Validation Loss vs. Trial')
plt.grid(True)
plt.tight_layout()
plt.show()

KeyError: 'loss'